# Using scvi-hub to upload pretrained scvi-tools models
In this tutorial, we will see how to use scvi-tools to upload pretrained models onto Hugging Face. We will also see how to handle large training datasets.

If you have not already, make sure to refer to our scvi_hub_into_and_download tutorial, which is a pre-requisite to this one. It introduces Hugging Face (HF) and the scvi-hub, and describes how to use them for downloading pre-trained models from the HF Model Hub.

# Imports
Let's start by adding the Python imports we need.

In [1]:
from scvi.hub import HubModel, HubMetadata, HubModelCardHelper

Global seed set to 0


# Model Card and Metadata

To upload pretrained models, you'll need to create an instance of the `HubModel` class and then simply call its `push_to_huggingface_hub` method.

As you can see from the API reference, the HubModel init function requires a HubMetadata and a Model Card. There are a few ways you can do this:
- The metadata can be either an instance of the `HubMetadata` class that contains the required metadata for this model, or a path to a JSON file on disk where this metadata can be read from.
- The Model Card can be either an instance of the `HubModelCardHelper` class created for this model, or an instance of the HF Model Card object, or a path to a Markdown file on disk where the model card can be read from.

In [5]:
hmo = HubModel.pull_from_huggingface_hub(
    repo_name="scvi-tools/demo-model",
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
hmo

HubModel with:
local_dir: /Users/valehvpa/.cache/huggingface/hub/models--scvi-tools--demo-model/snapshots/74bec019635e4155941c7e5353c33a7d7c0dabc6
model loaded? No
adata loaded? No
large_training_adata loaded? No
metadata:
HubMetadata(scvi_version='0.0.0', anndata_version='0.8.0', training_data_url=None, model_parent_module='scvi.model')
model_card:


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
license: cc-by-4.0                                                                                                 

library_name: scvi-tools                                                                                           

tags:                                                                                                              

 • model_cls_name:SCVI                                                                                             
 • scvi_version:0.0.0                                                                                              
 • anndata_version:0.8.0                                                                                           
 • modality:rna                                                                                                    
 • modality:protein                                                                                                
 • annotated:False                                                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
╔═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                   Description                                                   ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

This is a demo model trained on synthetic data.                                                                    

╔═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                Model properties                                                 ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

Many model properties are in the model tags. Some more are listed below.                                           

model_init_params:                                                                                                 

                                                                                                                   
{                                                                                                                  
                                                                                                                   
    "n_hidden": 128,                                                                                               
                                                                                                                   
    "n_latent": 10,                                                                                                
                                                                                                                   
    "n_layers": 1,                                                                                                 
                                                                                                                   
    "dropout_rate": 0.1,                                                                                           
                                                                                                                   
    "dispersion": "gene",                                                                                          
                                                                                                                   
    "gene_likelihood": "zinb",                                                                                     
                                               

Notice the local directory property `hmo.local_dir`. This where Hugging Face caches the data it downloads. Hugging Face uses a heuristic to determine the location (more [here](https://huggingface.co/docs/huggingface_hub/how-to-cache)). You can override it if you wish by calling:

`hmo = HubModel.pull_from_huggingface_hub(repo_name="scvi-tools/demo-model", cache_dir="/path/to/cache/dir")`

Next we can see the metadata and the model card. We also see that the model and data are not yet loaded into memory. We can load them by calling `hmo.model`, and `hmo.adata` respectively.

In [10]:
hmo.model

SCVI Model with the following params: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal
Training status: Trained
Latent model? False

In [9]:
hmo.adata

INFO     Reading adata...                                                                                          


AnnData object with n_obs × n_vars = 400 × 100
    obs: 'batch', 'labels', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'protein_names'
    obsm: 'protein_expression'